In [2]:
import numpy as np
import plotly.graph_objects as go
import quantfreedom as qf
from plotly.subplots import make_subplots

from quantfreedom.plotting.plot_helper_functions import plot_results_candles_and_chart

In [183]:
price_data = qf.generate_candles(
    number_of_candles=200,
    plot_candles=False,
    seed=None,
)
rsi_data = qf.from_talib(
    func_name="rsi",
    price_data=price_data,
    timeperiod=20,
    plot_results=False,
    # price="low",
)
df = price_data.droplevel(level=0, axis=1).loc['2001-08-10': '2001-11-10']
df['rsi'] = rsi_data['QuantFreedom'][20].loc['2001-08-10': '2001-11-10']
df = price_data.droplevel(level=0, axis=1)
df['rsi'] = rsi_data['QuantFreedom'][20]
price_low_values = df.low.values
price_close_values = df.close.values
price_open_values = df.open.values

piv_low_index = 0
piv_low = price_low_values[0]
piv_low_graph = np.full_like(price_low_values, np.nan)
piv_low_graph[0] = piv_low

rsi_values = df.rsi.values
divergence_index = np.full_like(price_low_values, -1, dtype=np.int_)
rsi_line_graph = np.full_like(price_low_values, np.nan)

candle_line_graph = np.full_like(price_low_values, np.nan)
candle_div_graph_lb = np.full_like(price_low_values, np.nan)
candle_div_graph_start = np.full_like(price_low_values, np.nan)

lookback = 30
plot_index = df.index
line_counter = 0

for x in range(1, price_low_values.size):
    if price_low_values[x] < piv_low and price_close_values[x] < price_open_values[x]:
        for lb in range(1, lookback):
            if x - lb > 0:
                lb_index = x - lb
                if (
                    rsi_values[x] > rsi_values[lb_index]
                    and price_close_values[lb_index] < price_open_values[lb_index]
                ):
                    candle_line_graph[line_counter + 1] = price_low_values[lb_index] - 2
                    candle_line_graph[line_counter] = price_low_values[x] - 2

                    rsi_line_graph[line_counter + 1] = rsi_values[lb_index] - 2
                    rsi_line_graph[line_counter] = rsi_values[x] - 2

                    divergence_index[line_counter + 1] = lb_index
                    divergence_index[line_counter] = x

                    line_counter += 2

                    break
            else:
                break
        piv_low = price_low_values[x]
        piv_low_graph[x] = price_low_values[x] - 2
        changed_pivot = True
    else:
        pass
    # keep some type of counter that checks if we have gone 20 candles without a new low
rsi_line_graph = rsi_line_graph[:line_counter]
candle_line_graph = candle_line_graph[:line_counter]
divergence_index = divergence_index[:line_counter]
divergence_index = plot_index[divergence_index.tolist()]
fig = make_subplots(
    rows=2,
    cols=1,
    row_heights=[0.7, 0.3],
    shared_xaxes=True,
    vertical_spacing=0.02,
)
fig.append_trace(
    go.Candlestick(
        x=plot_index,
        open=df["open"],
        high=df["high"],
        low=df["low"],
        close=df["close"],
    ),
    row=1,
    col=1,
)

fig.add_scatter(
    x=plot_index,
    y=piv_low_graph,
    mode="markers",
    marker=dict(size=4, color="MediumPurple"),
    name="candle pivot",
    row=1,
    col=1,
)

fig.append_trace(
    go.Scatter(
        x=plot_index,
        y=rsi_values,
        name="RSI",
    ),
    row=2,
    col=1,
)
start = 0
end = 2
for x in range(rsi_line_graph.size):
    fig.add_scatter(
        x=divergence_index[start:end],
        y=candle_line_graph[start:end],
        mode="markers+lines",
        marker=dict(size=10, symbol="arrow-up"),
        name="Candle Line Div",
        row=1,
        col=1,
        showlegend=False,
    )
    fig.add_scatter(
        x=divergence_index[start:end],
        y=rsi_line_graph[start:end],
        mode="markers+lines",
        marker=dict(size=10, symbol="arrow-up"),
        name="RSI div",
        row=2,
        col=1,
        showlegend=False,
    )
    start = end
    end += 2
fig.update_layout(
    xaxis_rangeslider_visible=False,
    height=800,
)
fig.show()